In [1]:
import pandas as pd
import matplotlib.pyplot as plt 
import numpy as np
import warnings
warnings.filterwarnings('ignore')
import pyarrow.parquet as pq 

In [2]:
import numpy as np
import tensorflow as tf 
import random

tf.random.set_seed(123)
np.random.seed(123)
random.seed(123)

from sklearn.model_selection import train_test_split

2023-08-05 11:08:34.872539: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
df_pred = pd.read_csv(r'/Users/prasanth/Downloads/pred_df.csv')
df_pred = df_pred.rename(columns={'premise': 'Predicted_french_premises', 'hypothesis': 'Predicted_french_hypothesis'})
df_pred.head()

,Predicted_french_premises,Predicted_french_hypothesis,label
0,Emily Ratajkowski. La carrière de mannequinage...,Emily Ratajkowski a modélisé pour les publicit...,0
1,National▁Recording▁Registry. Les enregistremen...,Red Headed▁Stranger est une œuvre.,0
2,Joan Crawford (née Lucille Fay LeSueur ; 23 ma...,Joan Crawford est allemande.,2
3,Metallica est un▁groupe de▁heavy▁metal américa...,Metallica est un▁groupe.,0
4,"John Stewart, connu▁sous le nom de Green▁Lante...",John Stewart est le premier super-héros afro-a...,2


In [4]:
filepath  = r'/Users/prasanth/Downloads/fr_fever-00000-of-00001-b7ec330d6224f90b.parquet'

In [5]:
table = pq.ParquetDataset(filepath)
df = table.read().to_pandas()

In [6]:
df_trans = df.drop(columns = ['label'], axis = 1)
df_trans.head()

,premise_original,hypothesis_original,premise,hypothesis
0,Islam . Sizable Muslim communities are also fo...,Islam does not have any followers.,Des communautés musulmanes considérables se tr...,L’islam n’a pas d’adeptes.
1,Gemini is a 2002 Indian Tamil-language action ...,Gemini is a movie that came out in 2002.,Gemini est un film d'action indien écrit et ré...,Gemini est un film sorti en 2002.
2,Theodore John Kaczynski ( [ kəˈzɪnski ] born M...,Ted Kaczynski is a serial arsonist.,Theodore John Kaczynski ( [ kznski ] né le 22 ...,Ted Kaczynski est un pyromane en série.
3,The Shield is an American crime drama televisi...,The Shield is an American military base.,The Shield est une série télévisée américaine ...,Le Bouclier est une base militaire américaine.
4,Hugh Jackman . His work in Les Misérables ear...,Hugh Jackman was in the film Les Misérables.,Hugh Jackman. Son travail dans Les Misérables ...,Hugh Jackman était dans le film Les Misérables.


In [7]:
X_prem = df['premise_original']  # the columns that are features
y_prem = df['premise']  # the target column

In [8]:
english_train, english_test, french_train, french_test = train_test_split(X_prem, y_prem, test_size=0.2, random_state=42)

In [10]:
english_test_df = english_test[:2000].to_frame('Original_english_premises')
english_test_df = english_test_df.reset_index(drop=True)
english_test_df.head()

,Original_english_premises
0,Emily Ratajkowski . Ratajkowski 's modeling ca...
1,National Recording Registry . The recordings p...
2,Joan Crawford ( born Lucille Fay LeSueur ; Mar...
3,Metallica is an American heavy metal band base...
4,"John Stewart , known as Green Lantern , is a f..."


In [11]:
df_new = pd.concat([english_test_df, df_pred], axis=1)

In [12]:
df_new.head()

,Original_english_premises,Predicted_french_premises,Predicted_french_hypothesis,label
0,Emily Ratajkowski . Ratajkowski 's modeling ca...,Emily Ratajkowski. La carrière de mannequinage...,Emily Ratajkowski a modélisé pour les publicit...,0
1,National Recording Registry . The recordings p...,National▁Recording▁Registry. Les enregistremen...,Red Headed▁Stranger est une œuvre.,0
2,Joan Crawford ( born Lucille Fay LeSueur ; Mar...,Joan Crawford (née Lucille Fay LeSueur ; 23 ma...,Joan Crawford est allemande.,2
3,Metallica is an American heavy metal band base...,Metallica est un▁groupe de▁heavy▁metal américa...,Metallica est un▁groupe.,0
4,"John Stewart , known as Green Lantern , is a f...","John Stewart, connu▁sous le nom de Green▁Lante...",John Stewart est le premier super-héros afro-a...,2


In [13]:
X_hypo = df['hypothesis_original']  # the columns that are features
y_hypo = df['hypothesis']  # the target column

In [14]:
english_train_hypo, english_test_hypo, french_train_hypo, french_test_hypo = train_test_split(X_hypo, y_hypo, test_size=0.2, random_state=42)

In [15]:
english_test_df1 = english_test_hypo[:2000].to_frame('Original_english_hypothesis')
english_test_df1 = english_test_df1.reset_index(drop=True)
english_test_df1.head()

,Original_english_hypothesis
0,Emily Ratajkowski modeled for retail advertise...
1,Red Headed Stranger is a work.
2,Joan Crawford is only German.
3,Metallica is a band.
4,John Stewart was the first African American su...


In [16]:
# Split the original df into two parts
df1 = df_new.iloc[:, :1]
df2 = df_new.iloc[:, 1:]

# Concatenate df1, english_test_df1, and df2 along the column axis
df = pd.concat([df1, english_test_df1, df2], axis=1)
df.head()

,Original_english_premises,Original_english_hypothesis,Predicted_french_premises,Predicted_french_hypothesis,label
0,Emily Ratajkowski . Ratajkowski 's modeling ca...,Emily Ratajkowski modeled for retail advertise...,Emily Ratajkowski. La carrière de mannequinage...,Emily Ratajkowski a modélisé pour les publicit...,0
1,National Recording Registry . The recordings p...,Red Headed Stranger is a work.,National▁Recording▁Registry. Les enregistremen...,Red Headed▁Stranger est une œuvre.,0
2,Joan Crawford ( born Lucille Fay LeSueur ; Mar...,Joan Crawford is only German.,Joan Crawford (née Lucille Fay LeSueur ; 23 ma...,Joan Crawford est allemande.,2
3,Metallica is an American heavy metal band base...,Metallica is a band.,Metallica est un▁groupe de▁heavy▁metal américa...,Metallica est un▁groupe.,0
4,"John Stewart , known as Green Lantern , is a f...",John Stewart was the first African American su...,"John Stewart, connu▁sous le nom de Green▁Lante...",John Stewart est le premier super-héros afro-a...,2


In [21]:
df.drop(['label'],axis = 1).head()

,Original_english_premises,Original_english_hypothesis,Predicted_french_premises,Predicted_french_hypothesis
0,Emily Ratajkowski . Ratajkowski 's modeling ca...,Emily Ratajkowski modeled for retail advertise...,Emily Ratajkowski. La carrière de mannequinage...,Emily Ratajkowski a modélisé pour les publicit...
1,National Recording Registry . The recordings p...,Red Headed Stranger is a work.,National▁Recording▁Registry. Les enregistremen...,Red Headed▁Stranger est une œuvre.
2,Joan Crawford ( born Lucille Fay LeSueur ; Mar...,Joan Crawford is only German.,Joan Crawford (née Lucille Fay LeSueur ; 23 ma...,Joan Crawford est allemande.
3,Metallica is an American heavy metal band base...,Metallica is a band.,Metallica est un▁groupe de▁heavy▁metal américa...,Metallica est un▁groupe.
4,"John Stewart , known as Green Lantern , is a f...",John Stewart was the first African American su...,"John Stewart, connu▁sous le nom de Green▁Lante...",John Stewart est le premier super-héros afro-a...


In [22]:
df.to_csv('final translated data.csv', index=False)


In [18]:
original_english_premises = df['Original_english_premises'].values
original_english_premises = original_english_premises.tolist()
original_english_hypothesis = df['Original_english_hypothesis'].values
original_english_hypothesis = original_english_hypothesis.tolist()


In [19]:
predicted_french_premises = df['Predicted_french_premises'].values
predicted_french_premises = predicted_french_premises.tolist()
predicted_french_hypothesis = df['Predicted_french_hypothesis'].values
predicted_french_hypothesis = predicted_french_hypothesis.tolist()

In [78]:
pip install textblob

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 636.8/636.8 kB 5.9 MB/s eta 0:00:00a 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [24]:
pip install tensorflow-hub

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.3/90.3 kB 4.2 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [27]:
import pandas as pd
from textblob import TextBlob
import tensorflow_hub as hub
from sklearn.metrics.pairwise import cosine_similarity

# Load the Universal Sentence Encoder's TF Hub module
embed = hub.load('https://tfhub.dev/google/universal-sentence-encoder/4')

# Assume we have a list of original hypotheses and their corresponding translations
#original_english_hypothesis = ['This is a good product', 'This is a bad product']
#predicted_french_hypothesis = ['C\'est un bon produit', 'C\'est un mauvais produit']

# Initialize a list to store the results
results = []

for original, translated in zip(original_english_hypothesis, predicted_french_hypothesis):
    # Sentiment analysis
    original_sentiment = TextBlob(original).sentiment.polarity
    translated_sentiment = TextBlob(translated).sentiment.polarity

    # Semantic similarity
    original_embedding = embed([original])
    translated_embedding = embed([translated])
    similarity = cosine_similarity(original_embedding, translated_embedding)

    # Store the results
    results.append([original, translated, original_sentiment, translated_sentiment, similarity[0][0]])

# Convert the results to a DataFrame
df_hypothesis = pd.DataFrame(results, columns=['Original', 'Translated', 'Original Sentiment', 'Translated Sentiment', 'Semantic Similarity'])

# Now you can perform various analyses on the DataFrame
average_original_sentiment = df_hypothesis['Original Sentiment'].mean()
average_translated_sentiment = df_hypothesis['Translated Sentiment'].mean()
average_semantic_similarity = df_hypothesis['Semantic Similarity'].mean()

print(f"Average original sentiment for hypothesis: {average_original_sentiment}")
print(f"Average translated sentiment for hypothesis: {average_translated_sentiment}")
print(f"Average semantic similarity for hypothesis: {average_semantic_similarity}")


Average original sentiment for hypothesis: 0.025420234788359808
Average translated sentiment for hypothesis: 0.00714859668109668
Average semantic similarity for hypothesis: 0.7578111886978149


In [29]:
import pandas as pd
from textblob import TextBlob
import tensorflow_hub as hub
from sklearn.metrics.pairwise import cosine_similarity

# Load the Universal Sentence Encoder's TF Hub module
embed = hub.load('https://tfhub.dev/google/universal-sentence-encoder/4')

# Assume we have a list of original hypotheses and their corresponding translations
#original_english_hypothesis = ['This is a good product', 'This is a bad product']
#predicted_french_hypothesis = ['C\'est un bon produit', 'C\'est un mauvais produit']

# Initialize a list to store the results
results = []

for original, translated in zip(original_english_premises, predicted_french_premises):
    # Sentiment analysis
    original_sentiment = TextBlob(original).sentiment.polarity
    translated_sentiment = TextBlob(translated).sentiment.polarity

    # Semantic similarity
    original_embedding = embed([original])
    translated_embedding = embed([translated])
    similarity = cosine_similarity(original_embedding, translated_embedding)

    # Store the results
    results.append([original, translated, original_sentiment, translated_sentiment, similarity[0][0]])

# Convert the results to a DataFrame
df_premises = pd.DataFrame(results, columns=['Original', 'Translated', 'Original Sentiment', 'Translated Sentiment', 'Semantic Similarity'])

# Now you can perform various analyses on the DataFrame
average_original_sentiment = df_premises['Original Sentiment'].mean()
average_translated_sentiment = df_premises['Translated Sentiment'].mean()
average_semantic_similarity = df_premises['Semantic Similarity'].mean()

print(f"Average original sentiment for premises: {average_original_sentiment}")
print(f"Average translated sentiment for premises: {average_translated_sentiment}")
print(f"Average semantic similarity for premises: {average_semantic_similarity}")



Average original sentiment for premises: 0.11384941907659496
Average translated sentiment for premises: 0.04092955928330926
Average semantic similarity for premises: 0.7736099362373352
